In [1]:
import secrets, base64, qrcode, time, struct, hmac, hashlib

### Generate a secret key

In [6]:
def generate_secret(length: int = 20):
    secret_bytes = secrets.token_bytes(length)
    #Convert to base32
    base32 = base64.b32encode(secret_bytes).decode('utf-8')
    return secret_bytes, base32

### Genrate a URI


In [8]:
secret_bytes, base32 = generate_secret()
print(secret_bytes, base32)

b'\xe2`\xae;S\xf35\xdb\t\xe0\xd3\xdf)\xdf\x04qiV\xc7\xa7' 4JQK4O2T6M25WCPA2PPSTXYEOFUVNR5H
